In [9]:
import cv2
import os

def extract_frame_from_video(video_path, frame_idx, output_path=None):
    """
    從影片中提取指定幀並儲存為圖片。
    
    Args:
        video_path (str): 影片路徑
        frame_idx (int): 想要提取的幀索引 (從 0 開始)
        output_path (str): 輸出圖片路徑，若為 None 則自動命名
        
    Returns:
        numpy.ndarray: 提取的幀影像，若失敗則回傳 None
    """
    # 檢查檔案是否存在
    if not os.path.exists(video_path):
        print(f"錯誤：找不到檔案 - {video_path}")
        return None
    
    # 開啟影片
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("錯誤：無法開啟影片")
        return None
    
    # 取得影片總幀數
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"影片總幀數: {total_frames}")
    
    # 檢查幀索引是否有效
    if frame_idx < 0 or frame_idx >= total_frames:
        print(f"錯誤：幀索引 {frame_idx} 超出範圍 [0, {total_frames-1}]")
        cap.release()
        return None
    
    # 跳轉到指定幀
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        print(f"錯誤：無法讀取第 {frame_idx} 幀")
        return None
    
    # 儲存圖片
    if output_path is None:
        output_path = f"frame_{frame_idx}.jpg"
    
    cv2.imwrite(output_path, frame)
    print(f"成功提取第 {frame_idx} 幀並儲存至: {output_path}")
    
    return frame


# 使用範例
if __name__ == "__main__":
    # 設定影片路徑和要提取的幀數
    video_file = r"C:\Users\f1410\Desktop\vicon_chessbord_img\20251125棋盤格\New Patient Classification\New Patient\New Session\court_2.2129991.overlay.mp4"
    target_frame = 168
    output_image = f"extracted_frame_{target_frame}.jpg"
    
    # 提取幀
    frame = extract_frame_from_video(video_file, target_frame, output_image)
    
    if frame is not None:
        print(f"影像尺寸: {frame.shape}")


影片總幀數: 1790
成功提取第 168 幀並儲存至: extracted_frame_168.jpg
影像尺寸: (1080, 1920, 3)


In [ ]:
'''
點選畫面，獲得該像素的RGB值
'''

import cv2
import numpy as np

def get_pixel_rgb_on_click(image_path):
    """
    顯示圖片並允許使用者點擊以獲取該像素的 RGB 值。
    
    Args:
        image_path (str): 圖片路徑
    """
    # 讀取圖片
    img = cv2.imread(image_path)
    if img is None:
        print(f"錯誤：無法讀取圖片 - {image_path}")
        return
    
    # 建立視窗
    window_name = "Click to get RGB - Press 'q' to quit"
    cv2.namedWindow(window_name)
    
    # 滑鼠回調函數
    def mouse_callback(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:  # 左鍵點擊
            # OpenCV 使用 BGR 格式，需轉換為 RGB
            bgr = img[y, x]
            rgb = bgr[::-1]  # BGR -> RGB
            
            print(f"座標: ({x}, {y}) | RGB: {tuple(rgb)} | BGR: {tuple(bgr)}")
            
            # 在圖片上標記點擊位置
            img_copy = img.copy()
            cv2.circle(img_copy, (x, y), 5, (0, 255, 0), -1)
            cv2.putText(img_copy, f"RGB: {tuple(rgb)}", (x + 10, y - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.imshow(window_name, img_copy)
    
    # 設定滑鼠回調
    cv2.setMouseCallback(window_name, mouse_callback)
    
    # 顯示圖片
    cv2.imshow(window_name, img)
    print("請點擊圖片上的任意位置以獲取 RGB 值，按 'q' 鍵退出")
    
    # 等待使用者操作
    while True:
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
    
    cv2.destroyAllWindows()


# 使用範例
if __name__ == "__main__":
    # 使用剛才提取的幀圖片
    image_file = f"extracted_frame_{target_frame}.jpg"
    get_pixel_rgb_on_click(image_file)


In [15]:
import cv2
import numpy as np

def get_pixel_coordinates_on_click(image_path):
    """
    點擊圖片以獲取像素座標。
    
    Args:
        image_path (str): 圖片路徑
    """
    # 讀取圖片
    img = cv2.imread(image_path)
    if img is None:
        print(f"錯誤：無法讀取圖片 - {image_path}")
        return
    
    # 建立視窗
    window_name = "Click to get Coordinates - Press 'q' to quit"
    cv2.namedWindow(window_name)
    
    # 滑鼠回調函數
    # 用於儲存當前滑鼠位置
    current_pos = {'x': 0, 'y': 0}
    
    def mouse_callback(event, x, y, flags, param):
        current_pos['x'] = x
        current_pos['y'] = y
        
        if event == cv2.EVENT_LBUTTONDOWN:  # 左鍵點擊
            print(f"像素座標: ({x}, {y})")
            
            # 在圖片上標記點擊位置
            img_copy = img.copy()
            cv2.circle(img_copy, (x, y), 5, (0, 255, 0), -1)
            cv2.putText(img_copy, f"({x}, {y})", (x + 10, y - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.imshow(window_name, img_copy)
        
        elif event == cv2.EVENT_MOUSEMOVE:  # 滑鼠移動時顯示放大鏡
            img_copy = img.copy()
            
            # 放大鏡參數
            zoom_factor = 3  # 放大倍數
            zoom_size = 50   # 放大區域的半徑
            magnifier_size = 150  # 放大鏡顯示窗口大小
            
            # 確保不超出圖片邊界
            y1 = max(0, y - zoom_size)
            y2 = min(img.shape[0], y + zoom_size)
            x1 = max(0, x - zoom_size)
            x2 = min(img.shape[1], x + zoom_size)
            
            # 提取放大區域
            roi = img[y1:y2, x1:x2]
            
            if roi.size > 0:
                # 放大區域
                zoomed = cv2.resize(roi, None, fx=zoom_factor, fy=zoom_factor, 
                                   interpolation=cv2.INTER_LINEAR)
                
                # 在放大鏡中心畫十字線
                center_y, center_x = zoomed.shape[0] // 2, zoomed.shape[1] // 2
                cv2.line(zoomed, (center_x - 10, center_y), (center_x + 10, center_y), 
                        (0, 255, 0), 1)
                cv2.line(zoomed, (center_x, center_y - 10), (center_x, center_y + 10), 
                        (0, 255, 0), 1)
                
                # 決定放大鏡顯示位置（避免遮擋滑鼠）
                mag_x = x + 30
                mag_y = y + 30
                
                # 確保放大鏡不超出圖片邊界
                if mag_x + magnifier_size > img.shape[1]:
                    mag_x = x - magnifier_size - 30
                if mag_y + magnifier_size > img.shape[0]:
                    mag_y = y - magnifier_size - 30
                
                # 調整放大鏡大小以適應顯示區域
                h, w = zoomed.shape[:2]
                if h > magnifier_size or w > magnifier_size:
                    scale = min(magnifier_size / h, magnifier_size / w)
                    zoomed = cv2.resize(zoomed, None, fx=scale, fy=scale)
                
                h, w = zoomed.shape[:2]
                
                # 確保有足夠空間顯示放大鏡
                if (mag_y >= 0 and mag_y + h <= img.shape[0] and 
                    mag_x >= 0 and mag_x + w <= img.shape[1]):
                    # 在圖片上疊加放大鏡
                    img_copy[mag_y:mag_y+h, mag_x:mag_x+w] = zoomed
                    # 畫放大鏡邊框
                    cv2.rectangle(img_copy, (mag_x, mag_y), (mag_x+w, mag_y+h), 
                                (255, 255, 0), 2)
            
            # 顯示當前座標
            cv2.putText(img_copy, f"({x}, {y})", (x + 10, y - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
            
            cv2.imshow(window_name, img_copy)
    
    # 設定滑鼠回調
    cv2.setMouseCallback(window_name, mouse_callback)
    
    # 顯示圖片
    cv2.imshow(window_name, img)
    print("請點擊圖片上的任意位置以獲取座標，按 'q' 鍵退出")
    
    # 等待使用者操作
    while True:
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
    
    cv2.destroyAllWindows()


# 使用範例
if __name__ == "__main__":
    # 使用剛才提取的幀圖片
    image_file = f"extracted_frame_{target_frame}.jpg"
    get_pixel_coordinates_on_click(image_file)


請點擊圖片上的任意位置以獲取座標，按 'q' 鍵退出
像素座標: (1904, 799)
像素座標: (1392, 485)
像素座標: (1345, 457)
像素座標: (1772, 801)
像素座標: (1324, 487)
像素座標: (1282, 457)
像素座標: (1010, 814)
像素座標: (924, 480)
像素座標: (916, 452)
像素座標: (122, 828)
像素座標: (493, 476)
像素座標: (524, 446)
像素座標: (413, 475)
像素座標: (451, 445)


In [31]:
import cv2
import numpy as np

def get_rgb_from_pixel(image_path, x, y):
    """
    從圖片中獲取指定像素座標的 RGB 值。
    
    Args:
        image_path (str): 圖片路徑
        x (int): 像素的 x 座標
        y (int): 像素的 y 座標
        
    Returns:
        tuple: (R, G, B) 值，若失敗則回傳 None
    """
    # 讀取圖片
    img = cv2.imread(image_path)
    
    if img is None:
        print(f"錯誤：無法讀取圖片 - {image_path}")
        return None
    
    # 檢查座標是否在圖片範圍內
    height, width = img.shape[:2]
    if not (0 <= x < width and 0 <= y < height):
        print(f"錯誤：座標 ({x}, {y}) 超出圖片範圍 (寬度: {width}, 高度: {height})")
        return None
    
    # OpenCV 使用 BGR 格式，需要轉換為 RGB
    bgr = img[y, x]  # 注意：OpenCV 使用 [row, col] = [y, x]
    rgb = (int(bgr[2]), int(bgr[1]), int(bgr[0]))  # BGR -> RGB
    
    return rgb


# 使用範例
if __name__ == "__main__":
    # 指定圖片路徑和座標
    image_file = f"extracted_frame_{target_frame}.jpg"
    pixel_x = 1904  # 替換為你想查詢的 x 座標
    pixel_y = 799  # 替換為你想查詢的 y 座標
    
    # 獲取 RGB 值
    rgb_value = get_rgb_from_pixel(image_file, pixel_x, pixel_y)
    
    if rgb_value is not None:
        print(f"座標 ({pixel_x}, {pixel_y}) 的 RGB 值為: {rgb_value}")
        print(f"R: {rgb_value[0]}, G: {rgb_value[1]}, B: {rgb_value[2]}")


座標 (1904, 799) 的 RGB 值為: (252, 251, 247)
R: 252, G: 251, B: 247


In [32]:
import cv2
import numpy as np

def find_pixels_by_rgb(image_path, target_rgb):
    """
    從圖片中找出所有符合指定 RGB 值的像素座標。
    
    Args:
        image_path (str): 圖片路徑
        target_rgb (tuple): 目標 RGB 值，例如 (230, 252, 79)
        
    Returns:
        numpy.ndarray: 符合條件的像素座標陣列 [[y1, x1], [y2, x2], ...]
                       若無結果或讀取失敗則回傳 None
    """
    # 讀取圖片
    img = cv2.imread(image_path)
    
    if img is None:
        print(f"錯誤：無法讀取圖片 - {image_path}")
        return None
    
    # OpenCV 使用 BGR 格式，需將 RGB 轉換為 BGR
    target_bgr = target_rgb[::-1]
    
    # 使用 NumPy 進行高效比對
    y_coords, x_coords = np.where(np.all(img == target_bgr, axis=-1))
    
    # 組合座標 (y, x)
    result_coords = np.column_stack((y_coords, x_coords))
    
    return result_coords


# 使用範例
if __name__ == "__main__":
    # 指定圖片路徑和目標 RGB 值
    image_file = f"extracted_frame_{target_frame}.jpg"
    target_color = (252, 251, 247)  # 替換為你想查詢的 RGB 值
    
    # 執行搜尋
    points = find_pixels_by_rgb(image_file, target_color)
    
    if points is not None and len(points) > 0:
        print(f"共找到 {len(points)} 個像素符合 RGB {target_color}。")
        print(f"前 10 個座標 (y, x):\n{points[:10]}")
    else:
        print("未找到符合的像素或執行失敗。")


共找到 3 個像素符合 RGB (252, 251, 247)。
前 10 個座標 (y, x):
[[ 129  190]
 [ 131  226]
 [ 799 1904]]
